In [7]:
import matplotlib.pyplot as plt
import numpy as np

import gpflow
import pandas as pd
import os
import matplotlib.pyplot as plt
import re
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm


In [8]:
model_data = pd.read_csv('/home/smmrrr/TEM_Analysis/TEM_Analysis/region_3_test_model_data_conif_vegc.csv')
unique_bins = sorted(model_data.stand_age_interval_min.unique())
bin_to_ordinal = {bin_val: idx for idx, bin_val in (enumerate(unique_bins))}
print(bin_to_ordinal)
model_data['ordinal_stand_age'] = model_data['stand_age_interval_min'].map(bin_to_ordinal)
model_data['ordinal_stand_age'].describe()
# model_data = model_data.loc[model_data['year']>=1900]

{0: 0, 5: 1, 10: 2, 15: 3, 20: 4, 25: 5, 30: 6, 35: 7, 40: 8, 45: 9, 50: 10, 55: 11, 60: 12, 65: 13, 70: 14, 75: 15, 80: 16, 85: 17, 90: 18, 95: 19, 100: 20, 125: 21, 150: 22}


count    287630.000000
mean          9.834551
std           7.873397
min           0.000000
25%           3.000000
50%           8.000000
75%          17.000000
max          22.000000
Name: ordinal_stand_age, dtype: float64

In [9]:
variables_to_compare = ['monthly_mean', 'tswrf_v11', 'tmp', 'precip', 'dtr', 'vpr', 'wind','co2','Ndep_Trendy']

base_comparison = model_data.loc[model_data['year'].between(1900,1950)].groupby(
    ['lon','lat','ordinal_stand_age'])[variables_to_compare].mean().reset_index()


model_data=model_data.merge(base_comparison, on = ['lon','lat','ordinal_stand_age'], suffixes = ('_ts', '_base'))
model_data=model_data.loc[model_data['monthly_mean_base']!=0]
# Iterate over the list of variables and create new columns in model_data for their ratios
for var in variables_to_compare:
    ts_column = f"{var}_ts"     # Time series data column name
    base_column = f"{var}_base" # Baseline data column name
    ratio_column = f"{var}_ratio" # Name for the new ratio column

    # Compute the ratio and create a new column in model_data
    if (var =='precip'):
        model_data[ratio_column] = model_data[ts_column] / model_data[base_column]
    else:
        model_data[ratio_column] = model_data[ts_column] - model_data[base_column]


In [10]:
model_data = model_data.loc[model_data['year']>=1950]
# model_data = model_data.loc[model_data['monthly_mean_ratio']<=3]

In [5]:
model_data.columns

Index(['lon', 'lat', 'year', 'stand_age_interval_min', 'silt_clay',
       'monthly_mean_ts', 'tswrf_v11_ts', 'tmp_ts', 'precip_ts', 'dtr_ts',
       'vpr_ts', 'wind_ts', 'Ndep_Trendy_ts', 'Nfer_crop', 'Nfer_pas', 'var',
       'Area', 'elev', 'region', 'co2_ts', 'ordinal_stand_age',
       'monthly_mean_base', 'tswrf_v11_base', 'tmp_base', 'precip_base',
       'dtr_base', 'vpr_base', 'wind_base', 'co2_base', 'Ndep_Trendy_base',
       'monthly_mean_ratio', 'tswrf_v11_ratio', 'tmp_ratio', 'precip_ratio',
       'dtr_ratio', 'vpr_ratio', 'wind_ratio', 'co2_ratio',
       'Ndep_Trendy_ratio'],
      dtype='object')

In [11]:
# Define the features and target variable
X = np.array(model_data[['tswrf_v11_ratio', 'tmp_ratio', 'precip_ratio',
       'dtr_ratio', 'vpr_ratio', 'wind_ratio', 'co2_ratio',
       'Ndep_Trendy_ratio']])
Y = np.array(model_data['monthly_mean_ratio'])
# Y = np.sqrt(model_data['monthly_mean_ratio'])

# Split the data into training and testing sets (80% train, 20% test)
# randomness can be controlled with a defined `random_state`
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
# Y = np.array(
#     model_data.loc[:, 'monthly_mean_ratio']
# )
# X = np.array(
#      # model_data.loc[model_data['year']>=2020,['ordinal_stand_age']]
#      model_data.loc[:,['ordinal_stand_age','tmp_ratio']]
# )
Y = Y.reshape(len(Y),1)

In [12]:
X_test

array([[-1.39633333e+00, -7.32333333e-01,  9.74761782e-01, ...,
         1.89666667e-01,  7.10766667e+00,  1.61782733e+02],
       [ 3.66235294e+00, -1.07235294e+00,  1.11086551e+00, ...,
         7.13725490e-02,  9.90470588e+00,  9.22240608e+01],
       [-1.47650000e+00,  3.84000000e-01,  1.33339549e+00, ...,
         6.55000000e-02,  8.72950000e+00,  1.05882280e+02],
       ...,
       [ 4.32137255e+00, -7.81960784e-01,  1.34725356e+00, ...,
        -4.96862745e-01,  4.72947059e+01,  2.18011094e+02],
       [ 1.12549020e+00,  8.86862745e-01,  7.88201217e-01, ...,
        -3.05294118e-01,  6.73647059e+01,  3.21865792e+02],
       [ 5.95509804e+00,  4.56078431e-01,  1.16322274e+00, ...,
        -3.81764706e-01,  7.96347059e+01,  2.81961261e+02]])

In [71]:
# rng = np.random.default_rng(1234)
# n_inducing = 500
# inducing_variable = rng.choice(X, size=n_inducing, replace=False)


In [13]:
rng = np.random.default_rng(1234)

# Calculate the percentiles (assuming X is sorted, if not, you need to sort it first)
# "percentiles" is an array of percentiles to calculate
percentiles = [10, 25, 50, 75, 90]
percentile_indices = np.percentile(np.arange(len(Y)), percentiles).astype(int)
percentile_indices

array([ 11160,  27901,  55802,  83703, 100444])

In [14]:
rng = np.random.default_rng(12345)
bin_size = 12
# Calculate the percentile values on Y
percentiles = [0, 10, 25, 50, 75, 90, 100]
percentile_values = np.percentile(Y_train, percentiles)
print(percentile_values)
# Initialize a list to store the indices of the subsets
subset_indices = np.array([])
# subset_indices= subset_indices.reshape((0, 1))
# For each pair of percentile values, select the indices of 100 points from within that range
for i in range(len(percentile_values) - 1):
    # Get the value range for the current bin
    low, high = percentile_values[i], percentile_values[i+1]
    
    # Find the indices where Y falls within the current percentile range 
    # Assuming Y is 1D and has the same length as the rows or columns of X
    in_range_indices = np.array(np.where((Y_train >= low) & (Y_train < high)))
    print(in_range_indices)
    in_range_indices=in_range_indices.reshape(in_range_indices.shape[1],1)
    # Randomly select up to 100 indices within this range
    # Here we are careful to only take as many indices as are available if there are fewer than 100
    selected_indices = rng.choice(in_range_indices, size= bin_size, replace=False)
    # selected_indices=selected_indices.reshape()
    # Extend the subset_indices list with the selected indices
    if (i==0):
        subset_indices=selected_indices
    else:
        subset_indices=np.vstack([subset_indices,selected_indices])
# subset_indices.shape
# Using the selected indices, we can now reference the corresponding rows from X
inducing_variable = X_train[subset_indices.flatten(),:]

[-10868.54         -713.48034938    -32.9154902     537.84726471
   1636.28323529   2995.20815686  15387.19214286]
[[    0    28    29 ... 78117 78120 78121]]
[[   16    24    32 ... 78106 78108 78118]]
[[    7     8    12 ... 78113 78116 78122]]
[[    1     2     3 ... 78114 78115 78119]]
[[    4     9    11 ... 78109 78111 78123]]
[[    5    21    25 ... 78058 78062 78064]]


In [6]:
# inducing_variable

In [15]:
model = gpflow.models.SGPR(
    (X, Y),
    kernel=gpflow.kernels.RBF(),
    # kernel=gpflow.kernels.SquaredExponential(),
    inducing_variable=inducing_variable,
        # noise_variance=1.0  # Set to some reasonable initial value
)

2024-04-18 14:52:46.017934: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-04-18 14:52:46.018010: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bm4): /proc/driver/nvidia/version does not exist
2024-04-18 14:52:46.022035: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
print(gpflow.utilities.print_summary(model, "notebook"))

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,1.0
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,1.0
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,1.0
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(72, 8)",float64,"[[-1.29500000e+00, 1.20389000e+00, 9.85975007e-01..."


None


In [18]:
import tensorflow as tf

start_time = time.time()
# Instantiate an Adam optimizer with a set learning rate
adam_optimizer = tf.optimizers.Adam(learning_rate=0.1)

# Run the optimization loop
# Perform a number of optimization steps
num_iterations = 200  # Choose an appropriate number of iterations for your problem
for i in range(num_iterations):
    adam_optimizer.minimize(model.training_loss, model.trainable_variables)
    if (i % 100 ==0):
        print(i)
        elapsed_time_minutes = (time.time() - start_time) / 60
        print(f"Elapsed time: {elapsed_time_minutes:.2f} minutes")

0
Elapsed time: 0.02 minutes
100
Elapsed time: 2.16 minutes


In [31]:
X_train.shape[1]

8

In [32]:
print(gpflow.utilities.print_summary(model, "notebook"))
###save model
model.compiled_predict_f = tf.function(
    lambda Xnew: model.predict_f(Xnew, full_cov=False),
    input_signature=[tf.TensorSpec(shape=[None, X_train.shape[1]], dtype=tf.float64)],
)
model.compiled_predict_y = tf.function(
    lambda Xnew: model.predict_y(Xnew, full_cov=False),
    input_signature=[tf.TensorSpec(shape=[None, X_train.shape[1]], dtype=tf.float64)],
)

save_dir = "/home/smmrrr/TEM_Analysis/TEM_Analysis/models/"
save_name = "test_gp"
tf.saved_model.save(model, save_dir+save_name)


name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,10.20754
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,12.64812
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,7.36865
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(72, 8)",float64,"[[-2.38590000e+00, -8.24101000e+00, -4.74807000e+00..."


None
INFO:tensorflow:Assets written to: /home/smmrrr/TEM_Analysis/TEM_Analysis/models/test_gp/assets


In [28]:
# loaded_model = tf.saved_model.load( save_dir+save_name)

In [29]:
mean, var =  model.predict_y(
X_test
)

Y_pred = mean.numpy().flatten()
# Y_test = np.array(model_data.loc[model_data['year'].between(1950,2020),'monthly_mean_ratio'])

# Calculate the sum of squares of residuals
SS_res = np.sum((Y_test - Y_pred) ** 2)

# Calculate the total sum of squares
SS_tot = np.sum((Y_test - np.mean(Y_test)) ** 2)
rmse = np.sqrt(SS_res/len(Y_test))
# Calculate the R² score
r2_score = 1 - (SS_res / SS_tot)
print("Test RMSE: ",rmse)
print("Test R2: ",r2_score)



Test RMSE:  1810.696518832884
Test R2:  0.060971960982251994


In [20]:
mean, var = model.predict_y(
#     np.array(
#      model_data.loc[model_data['year'].between(1950,2020),[['tmp_ratio', 'co2_ratio', 'ordinal_stand_age']]]
# )
X_train
)

Y_pred = mean.numpy().flatten()
# Y_test = np.array(model_data.loc[model_data['year'].between(1950,2020),'monthly_mean_ratio'])

# Calculate the sum of squares of residuals
SS_res = np.sum((Y_train - Y_pred) ** 2)

# Calculate the total sum of squares
SS_tot = np.sum((Y_train - np.mean(Y_train)) ** 2)
rmse = np.sqrt(SS_res/len(Y_test))
# Calculate the R² score
r2_score = 1 - (SS_res / SS_tot)
print("Train RMSE: ",rmse)
print("Train R2: ",r2_score)



Train RMSE:  2753.2375472308922
Train R2:  0.06238171334554565
